In [1]:
# save as: export_recos_top3_only.py
import os, re, time, logging
from datetime import datetime, timezone
from typing import Optional, Dict, Any, List

import psycopg2
import pandas as pd
from dotenv import load_dotenv

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import csv, unicodedata

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
load_dotenv()

# ======= CONFIG (edit if needed) =======
PREDICTIONS_TABLE = '"Prediction".sample_recommendation_data'
POLICY_NO_COL      = '"policy no"'
STATUS_COL         = '"Predicted Status"'
TOP3_REASONS_COL   = '"Top 3 Reasons"'

MAX_ROWS       = int(os.getenv("MAX_ROWS", "0"))          # 0 = all
SLEEP_BETWEEN  = float(os.getenv("SLEEP_BETWEEN", "0.05"))


from langchain_azure_ai.chat_models.inference import AzureAIChatCompletionsModel
AZURE_INFERENCE_API_KEY   = os.getenv("AZURE_INFERENCE_API_KEY")
AZURE_INFERENCE_MODEL     = os.getenv("AZURE_INFERENCE_MODEL")
AZURE_INFERENCE_ENDPOINT     = os.getenv("AZURE_INFERENCE_ENDPOINT")
LLM_TEMPERATURE  = float(os.getenv("LLM_TEMPERATURE", "0.0"))

# DB
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT", "5432")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# ======= DB helpers =======
def get_conn():
    return psycopg2.connect(
        host=DB_HOST, port=DB_PORT, dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD
    )

def df_read_sql(sql: str, params: tuple | None = None) -> pd.DataFrame:
    with get_conn() as conn:
        return pd.read_sql(sql, conn, params=params)

def exec_fetch_one(conn, sql: str, params: tuple | None = None) -> Optional[Dict[str, Any]]:
    with conn.cursor() as cur:
        cur.execute(sql, params or ())
        row = cur.fetchone()
        if not row:
            return None
        cols = [d[0] for d in cur.description]
    return dict(zip(cols, row))


# ======= Normalization & SQL bits =======
def normalize_policy_value(p: Any) -> str:
    s = str(p or "").strip()
    if s.startswith(("'", '"')):
        s = s[1:]
    return s.replace(" ", "").upper()

def norm_policy_expr(quoted_col: str) -> str:
    return f"UPPER(REPLACE(REGEXP_REPLACE({quoted_col}::text, '^''', ''), ' ', ''))"

# ======= Tiny schema RAG =======
def build_schema_docs(sample_rows: int = 25) -> FAISS:
    try:
        sample_df = df_read_sql(f"SELECT * FROM {PREDICTIONS_TABLE} LIMIT {sample_rows};")
    except Exception as e:
        logging.warning("Schema sample failed; falling back to minimal docs: %s", e)
        sample_df = pd.DataFrame()

    texts: List[str] = []
    if not sample_df.empty:
        for col in sample_df.columns:
            val = ""
            try:
                val = sample_df[col].dropna().astype(str).iloc[0][:160]
            except Exception:
                pass
            desc = f'{col}: column in {PREDICTIONS_TABLE}'
            if val:
                desc += f" | sample_value={val}"
            texts.append(desc)
    else:
        texts = [
            f'{POLICY_NO_COL}: column in {PREDICTIONS_TABLE}',
            f'{STATUS_COL}: column in {PREDICTIONS_TABLE}',
            f'{TOP3_REASONS_COL}: column in {PREDICTIONS_TABLE}',
        ]

    docs = [Document(page_content=t) for t in texts]
    embeds = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
    return FAISS.from_documents(docs, embeds)

schema_vs = build_schema_docs()
schema_retriever = schema_vs.as_retriever(search_kwargs={"k": 8})
def get_schema_snippets(q: str) -> str:
    docs = schema_retriever.get_relevant_documents(q)
    return "\n".join(d.page_content for d in docs)

# ======= LLM client =======
llm = AzureAIChatCompletionsModel(endpoint=AZURE_INFERENCE_ENDPOINT, model=AZURE_INFERENCE_MODEL, credential=AZURE_INFERENCE_API_KEY, temperature=LLM_TEMPERATURE)

def generate_sql_with_llm(policy_no_value: Any) -> str:
    schema_snips = get_schema_snippets(
        f"Fetch row by {POLICY_NO_COL} for policy {policy_no_value}"
    )
    prompt = f"""
You write ONE valid PostgreSQL SELECT query for table {PREDICTIONS_TABLE}. No markdown.

Rules:
- Table: {PREDICTIONS_TABLE} only.
- Return all columns (*).
- Filter by "policy no" exactly.
- Note: some policy numbers start with a single quote, e.g. '2011200... 
  So you must escape it correctly using three single quotes in SQL.
- Example:
  SELECT * FROM "Prediction".sample_recommendation_data 
  WHERE "policy no" = '''201120010123703748704000';
- End the query with a semicolon.

Fetch the entire row for policy value {policy_no_value}.
"""
    draft = llm.invoke(prompt).content.strip()

    # extract just the query (ignore markdown or text)
    m = re.search(r"(SELECT\s.+?;)", draft, flags=re.IGNORECASE | re.DOTALL)
    sql = m.group(1).strip() if m else draft

    # Safety: add semicolon if missing
    if not sql.endswith(";"):
        sql += ";"

    return sql

# ======= Status helpers =======
def is_not_renewed(v: Any) -> bool:
    s = str(v or "").strip().strip('"').strip("'").lower()
    return s in {"not renewed", "not_renewed", "churn", "no", "false"}

def is_renewed(v: Any) -> bool:
    s = str(v or "").strip().strip('"').strip("'").lower()
    return s in {"renewed", "yes", "true"}

# ======= Recommendation (Top 3 Reasons ONLY) =======
def extract_top3_reasons(row: Dict[str, Any]) -> str:
    # Handle exact and case-variant keys safely
    for k in (TOP3_REASONS_COL.strip('"'), TOP3_REASONS_COL, TOP3_REASONS_COL.lower()):
        if k in row:
            return str(row[k] or "").strip()
    # try looser match
    for key in row.keys():
        if key.lower() == TOP3_REASONS_COL.strip('"').lower():
            return str(row[key] or "").strip()
    return ""

def generate_recommendation_from_top3(top3_text: str) -> str:
    if not top3_text:
        return ""  # nothing to say if no reasons present
    prompt = f"""
You are a professional customer retention strategist for a car insurance company.

Your job is to analyze the situation based only on the provided reasons text and produce a clear, helpful recommendation to improve retention.

Guidelines:
- Carefully analyze all churn reasons provided; the recommendation must address each reason, not just one.
- For each reason, include a concrete counter-strategy or benefit that directly mitigates it.
- If suggesting discounts, keep them practical (10–30%); never exceed 30%. Prefer non-price actions when appropriate.
- Do not invent new reasons or reference any data beyond the input reasons.

Style rules:
- Deliver a concise third-person strategy note — never an email.
- Do not address the customer directly (“you”, “we”, “please”).
- Keep the tone professional, helpful, and concise.
- Output clear bullet points (3–4), each focused on a distinct action or insight.
- Return only bullet points, and each bullet **must start with "• "**.

Top 3 Reasons:
{top3_text}

Return only bullet points.
""".strip()
    return llm.invoke(prompt).content.strip()

# ======= Pipeline =======
def load_full_table() -> pd.DataFrame:
    df = df_read_sql(f"SELECT * FROM {PREDICTIONS_TABLE};")
    if MAX_ROWS > 0 and len(df) > MAX_ROWS:
        df = df.head(MAX_ROWS)
    return df

def _clean_for_excel(x: object) -> str:
    s = "" if x is None else str(x)
    s = unicodedata.normalize("NFC", s)
    
    # Normalize bullet styles
    replacements = {
        "â€¢": "•",   # Fix mojibake
        "*": "•",    # Convert stars
        "- ": "• ",  # Convert dashes followed by space
        "·": "•",    # Common mid-dot variants
        "• ": "• ",  # Keep consistent spacing
    }
    for old, new in replacements.items():
        s = s.replace(old, new)
    
    # Normalize newlines for Excel
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    return s.strip()

def main():
    df = load_full_table()

    # sanity for needed columns
    if POLICY_NO_COL.strip('"') not in [c.strip('"') for c in df.columns]:
        raise ValueError(f"Policy column {POLICY_NO_COL} not found.")
    if STATUS_COL.strip('"') not in [c.strip('"') for c in df.columns]:
        logging.warning("Status column %s not found; treating all rows as NOT RENEWED.", STATUS_COL)

    out_rows: List[Dict[str, Any]] = []

    with get_conn() as conn:   # ✅ only one connection for the whole run
        for idx, src_row in df.iterrows():
            policy_val = src_row[POLICY_NO_COL.strip('"')] if POLICY_NO_COL.strip('"') in df.columns else src_row.get(POLICY_NO_COL, "")
            status_val = src_row[STATUS_COL.strip('"')] if STATUS_COL.strip('"') in df.columns else src_row.get(STATUS_COL, "")

            try:
                # --- Only "Not Renewed" rows go to LLM
                if not is_not_renewed(status_val):
                    row_out = src_row.to_dict()
                    row_out["recommendation_text"] = ""
                    out_rows.append(row_out)
                    continue

                # ✅ use the same open connection
                sql = generate_sql_with_llm(policy_val)
                fetched = exec_fetch_one(conn, sql)

                if not fetched:
                    logging.warning("No row fetched for policy=%s; leaving blank.", policy_val)
                    row_out = src_row.to_dict()
                    row_out["recommendation_text"] = ""
                    out_rows.append(row_out)
                    continue

                # Extract ONLY Top 3 Reasons and generate reco purely from it
                top3 = extract_top3_reasons(fetched)
                reco = generate_recommendation_from_top3(top3)

                row_out = src_row.to_dict()
                row_out["recommendation_text"] = reco
                out_rows.append(row_out)

                if SLEEP_BETWEEN > 0:
                    time.sleep(SLEEP_BETWEEN)
                if (idx + 1) % 25 == 0:
                    logging.info("Processed %d/%d rows", idx + 1, len(df))

            except Exception as e:
                logging.exception("Failed policy=%s: %s", policy_val, e)
                row_out = src_row.to_dict()
                row_out["recommendation_text"] = ""
                out_rows.append(row_out)

    # ======= After loop =======
    if not out_rows:
        logging.warning("No data to export.")
        return

    first_cols = [c for c in out_rows[0].keys() if c != "recommendation_text"]
    cols = first_cols + ["recommendation_text"]
    df_out = pd.DataFrame(out_rows)[cols]

    # clean the recommendation column before saving
    if "recommendation_text" in df_out.columns:
        df_out["recommendation_text"] = df_out["recommendation_text"].map(_clean_for_excel)

    ts = datetime.now(timezone.utc).astimezone().strftime("%Y%m%d_%H%M%S")
    out_dir = os.path.join(os.getcwd(), "out")
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, f"predictions_recommendations_top3_{ts}.csv")

    # BOM + QUOTE_ALL so bullets/newlines stay inside a single Excel cell
    df_out.to_csv(
        out_path,
        index=False,
        encoding="utf-8-sig",
        quoting=csv.QUOTE_ALL,
        lineterminator="\n",
    )
    logging.info("CSV written: %s (rows=%d)", out_path, len(df_out))

if __name__ == "__main__":
    main()

C:\Users\Kaviyam\AppData\Local\Temp\ipykernel_9440\2459612388.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn, params=params)
C:\Users\Kaviyam\AppData\Local\Temp\ipykernel_9440\2459612388.py:99: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeds = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
2025-10-15 16:16:54,968 | INFO | Use pytorch device_name: cuda:0
2025-10-15 16:16:54,968 | INFO | Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
202